In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
def agente_buscador(topico):

         buscador = Agent(
             name="agente_buscador",
             model="gemini-2.0-flash",
             description="Agente que busca informações no Google",
             tools=[google_search],
             instruction="""Você é um assistente de cálculo. A sua tarefa é usar a ferramenta de busca do google (Google Search)
             para calculara melhor forma de organizar uma planinha de gastos .
             Foque em encontrar uma forma de reduzir gastos e organizar os custos em tópicos: essenciais, importantes, segundo plano
             e por ultimo descartáveis.
             """


         )

         entrada_do_agente_buscador = f"Tópico: {topico}"

         lancamentos = call_agent(buscador, entrada_do_agente_buscador)
         return lancamentos


In [8]:
def agente_planejador(topico, dados_matematicos):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um planejador de planilha, especialista em organização de renda. Com base na lista de
        custos que vai de essenciais até descártaveis matemático, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevates que poderiamos destacar na planilha. Você também pode usar
        o (google_search) para encontrar mais
        informações sobre o tema e aprofundar.
        Ao final, você irá escolher o melhor modo de organizar a renda para minimizar os custos e
        máximizar a renda.
        """,
        description="Agente que planeja panilhas",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {dados_matematicos}"
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
def agente_criador(topico, formacao_da_planilha):
    criador = Agent(
        name="agente_criador",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um criador de planilhas, irá usar os dados coletados para montar a planilha da melhor
            forma, vizando o máximo de lucro para a pessoa.
            Irá deixar um espaço no final da planilha e deixar três tópicos de melhoria e ações para serem feitas
            além das contas matemáticas.
            """,
        description="Agente criador de planilhas"
    )
    entrada_do_agente_criador = f"Tópico: {topico}\nPlano de post: {formacao_da_planilhat}"
    rascunho = call_agent(criador, entrada_do_agente_criador)
    return rascunho

In [9]:
def agente_finalizador(topico, rascunho_gerado):
    finalizador = Agent(
        name="agente_finalizador",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Revizor de Conteúdo super meticuloso, especializado em planos financeiros, com foco em achar o melhor
            plano financeiro.
            Use um tom calmo e paciente, sempre vizando o bem estar do seu consultor.
            Revise o rascunho de planilha de renda abaixo, verificando clareza, concisão, correção, coesão, funcionalidade e tom.
            Se o rascunho estiver bom, responda apenas 'A plnilha está ótima e pronta para ser aplicado!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente finalizador de planilhas financeiras."
    )
    entrada_do_agente_finalizador = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    planilha_finalizada = call_agent(finalizador, entrada_do_agente_finalizador)
    return planilha_finalizada

In [24]:
print("❦ 𝐈𝐧𝐢𝐜𝐢𝐚𝐧𝐝𝐨 𝐩𝐞𝐬𝐪𝐮𝐢𝐬𝐚 𝐝𝐚 𝐬𝐮𝐚 𝐩𝐥𝐚𝐧𝐢𝐥𝐡𝐚! ❦")

topico = input("❓ Por favor, digite seus ganhos, seus gastos e de onde eles vêm")

if not topico:
  print("você não digitou nenhum dado. Por favor, tente novamente para podermos te ajudar.")
else:
  print(f"Maravilha! Vamos então pesquisar a melhor forma de organizar seus dados em {topico}")

  lancamentos_buscados = agente_buscador(topico)
  print("\n--- 💸 𝐃𝐚𝐝𝐨𝐬 𝐝𝐨 𝐚𝐠𝐞𝐧𝐭𝐞 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("───────────────────────ೋღ 🌺 ღೋ───────────────────────")

  plano_de_planilha = agente_planejador(topico, lancamentos_buscados)
  print("\n--- 📃 𝐑𝐞𝐬𝐮𝐥𝐭𝐚𝐝𝐨𝐬 𝐝𝐨 𝐚𝐠𝐞𝐧𝐭𝐞 𝟐 (planejador) ---\n")
  display(to_markdown(plano_de_planilha))
  print("───────────────────────ೋღ 🌺 ღೋ───────────────────────")

  rascunho_de_planilha = agente_criador(topico, formacao_da_planilha)
  print("\n--- 📤 𝐑𝐞𝐬𝐮𝐥𝐭𝐚𝐝𝐨 𝐝𝐨 𝐚𝐠𝐞𝐧𝐭𝐞 𝟑 (criador) ---\n")
  display(to_markdown(rascunho_de_planilha))

  planilha_finalizada = agente_finalizador(topico, rascunho_gerado)
  print("\n--- 💎 𝐑𝐞𝐬𝐮𝐥𝐭𝐚𝐝𝐨 𝐝𝐨 𝐚𝐠𝐞𝐧𝐭𝐞 𝟒 (finalizador) ---\n")
  display(to_markdown(planilha_finalizada))



❦ 𝐈𝐧𝐢𝐜𝐢𝐚𝐧𝐝𝐨 𝐩𝐞𝐬𝐪𝐮𝐢𝐬𝐚 𝐝𝐚 𝐬𝐮𝐚 𝐩𝐥𝐚𝐧𝐢𝐥𝐡𝐚! ❦
❓ Por favor, digite seus ganhos, seus gastos e de onde eles vêmOlá! Sou o Pedro, tenho 25 anos e estou começando a organizar minhas finanças.  Minha renda principal vem do meu trabalho como analista júnior em uma empresa, onde ganho cerca de R$ 3.500. Além disso, faço alguns trabalhos freelancers como tradutor nas horas vagas, o que me rende uma média de R$ 500 por mês. Somando tudo, minha renda mensal total fica em torno de R$ 4.000.  Agora, sobre onde meu dinheiro vai:  Moradia: Moro em um apartamento dividido com um amigo, então meu aluguel é R$ 800. As contas da casa (luz, água, gás, condomínio) somam cerca de R$ 350 para a minha parte. Alimentação: Gasto em média R$ 600 no mercado e cerca de R$ 300 com refeições fora de casa ou pedidos por aplicativo durante a semana. Transporte: Uso transporte público e, ocasionalmente, aplicativos de carro. Gasto cerca de R$ 250 por mês. Serviços e Assinaturas: Minha internet e plano de celular custam junt

NameError: name 'Agent' is not defined